In [1]:
import tensorflow as tf
import wandb
from wandb.integration.keras import WandbMetricsLogger
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python import PythonOperator
import numpy as np


2024-11-12 14:28:36.472555: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 14:28:36.475305: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 14:28:36.483348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731389316.498013 2926522 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731389316.502114 2926522 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 14:28:36.515620: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

/home/aiffel04/my_project/aiffel/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

In [3]:
tf.__version__

'2.18.0'

In [4]:
# Wandb 설정

wandb.login()
WANDB_PROJECT = "text"
WANDB_ENTITY ="ctan0722-innodigital"

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jbw8715 (jbw8715-chung-ang-university). Use `wandb login --relogin` to force relogin


In [5]:
# 1st
import tensorflow as tf

def load_and_preprocess_data():
    # TensorFlow의 Keras API를 사용하여 MNIST 데이터셋 로드
    #[[YOUR CODE]]
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    
    # 데이터 정규화 (픽셀 값을 0과 1 사이로 변환)
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    
    # 차원 추가 (CNN에 사용하기 위해 (28, 28) -> (28, 28, 1)로 변환)
    x_train = x_train[..., tf.newaxis]
    x_test = x_test[..., tf.newaxis]
    
    return (x_train, y_train), (x_test, y_test)



In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_model(learning_rate=0.001, conv1_filters=32, conv2_filters=64):
    """CNN 모델 생성"""
    model = models.Sequential([
        # 첫 번째 합성곱 레이어
        layers.Conv2D(conv1_filters, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        
        # 두 번째 합성곱 레이어
        layers.Conv2D(conv2_filters, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # 완전 연결층을 위한 플래튼 레이어
        layers.Flatten(),
        
        # 완전 연결층
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')  # MNIST 데이터셋의 10개 클래스
    ])
    
    # 모델 컴파일
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [7]:
import wandb
from datetime import datetime

def train_model(**context):
    """모델 학습 및 W&B 로깅"""
    # W&B 초기화
    wandb.init(
        project="mnist_classification",  # 프로젝트 이름
        config={
            "learning_rate": 0.001,
            "conv1_filters": 32,
            "conv2_filters": 64,
            "epochs": 5,
            "batch_size": 64
        }
    )
    
    # 데이터 로드
    (x_train, y_train), (x_test, y_test) = load_and_preprocess_data()
    
    # 모델 생성
    config = wandb.config
    model = create_model(
        learning_rate=config.learning_rate,
        conv1_filters=config.conv1_filters,
        conv2_filters=config.conv2_filters
    )
    
    # 모델 학습
    history = model.fit(
        x_train, y_train,
        epochs=config.epochs,
        batch_size=config.batch_size,
        validation_data=(x_test, y_test),
        callbacks=[wandb.keras.WandbCallback(save_graph=False, save_model=False)]
    )
    
    # 모델 평가
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {test_accuracy}")
    
    # W&B에 최종 메트릭 기록 (test_loss, test_accuracy)
    wandb.log({
        "test_loss": test_loss,
        "test_accuracy": test_accuracy
    })
    
    # 모델 저장
    model.save(f"mnist_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.keras")
    
    wandb.finish()


In [8]:
def hyperparameter_sweep():
    """W&B를 사용한 하이퍼파라미터 튜닝"""
    sweep_config = {
        'method': 'random',
        'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
        'parameters': {
            'learning_rate': {'values': [0.001, 0.01, 0.0001]},
            'conv1_filters': {'values': [16, 32, 64]},
            'conv2_filters': {'values': [32, 64, 128]},
            'batch_size': {'values': [64, 128, 256]}
        }
    }
    
    sweep_id = wandb.sweep(sweep_config, project="text")
    wandb.agent(sweep_id, train_model, count=5)

In [9]:
# Airflow DAG 정의

import pendulum  # Airflow는 pendulum을 사용합니다

local_tz = pendulum.timezone("Asia/Seoul")  # 한국 시간대 사용

default_args = {
    'owner': 'airflow',
    'start_date': datetime(2024, 1, 1, tzinfo=local_tz),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}



dag = DAG(
    'mnist_training_pipeline',
    default_args=default_args,
    description='MNIST 학습 파이프라인',
    schedule = '@daily',
    catchup=False
)

# DAG 태스크 정의
preprocessing_task = PythonOperator(
    task_id='load_and_preprocess_data',
    python_callable=load_and_preprocess_data,
    dag=dag
)

training_task = PythonOperator(
    task_id='train_model',
    python_callable=train_model,
    dag=dag
)

hyperparameter_tuning_task = PythonOperator(
    task_id='hyperparameter_tuning',
    python_callable=hyperparameter_sweep,
    dag=dag
)

# 태스크 의존성 설정
preprocessing_task >> training_task >> hyperparameter_tuning_task

# Jupyter Notebook에서 직접 실행하기 위한 코드
if __name__ == "__main__":
    # 단일 실험 실행
    train_model()

    hyperparameter_sweep()

/home/aiffel04/my_project/aiffel/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107 UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

2024-11-12 14:34:56.981308: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.8639 - loss: 0.4473 - val_accuracy: 0.9838 - val_loss: 0.0558
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.9817 - loss: 0.0593 - val_accuracy: 0.9886 - val_loss: 0.0372
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.9878 - loss: 0.0409 - val_accuracy: 0.9887 - val_loss: 0.0351
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.9907 - loss: 0.0297 - val_accuracy: 0.9845 - val_loss: 0.0454
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9923 - loss: 0.0245 - val_accuracy: 0.9892 - val_loss: 0.0299
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9863 - loss: 0.0387
Test accuracy: 0.9891999959945679


Create sweep with ID: sxu9u2ji
Sweep URL: https://wandb.ai/jbw8715-chung-ang-university/text/sweeps/sxu9u2ji


wandb: Agent Starting Run: j6jjsh56 with config:
wandb: 	batch_size: 128
wandb: 	conv1_filters: 16
wandb: 	conv2_filters: 32
wandb: 	learning_rate: 0.001
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

/home/aiffel04/my_project/aiffel/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107 UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

Epoch 1/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7993 - loss: 0.7172 - val_accuracy: 0.9749 - val_loss: 0.0834
Epoch 2/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.9724 - loss: 0.0884 - val_accuracy: 0.9833 - val_loss: 0.0530
Epoch 3/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.9812 - loss: 0.0594 - val_accuracy: 0.9865 - val_loss: 0.0453
Epoch 4/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9863 - loss: 0.0457 - val_accuracy: 0.9871 - val_loss: 0.0412
Epoch 5/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.9885 - loss: 0.0394 - val_accuracy: 0.9882 - val_loss: 0.0371
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9846 - loss: 0.0460
Test accuracy: 0.9882000088691711


wandb: Agent Starting Run: eklpj49y with config:
wandb: 	batch_size: 64
wandb: 	conv1_filters: 32
wandb: 	conv2_filters: 64
wandb: 	learning_rate: 0.01
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


/home/aiffel04/my_project/aiffel/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107 UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9069 - loss: 0.2862 - val_accuracy: 0.9794 - val_loss: 0.0649
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9817 - loss: 0.0585 - val_accuracy: 0.9793 - val_loss: 0.0696
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9863 - loss: 0.0473 - val_accuracy: 0.9839 - val_loss: 0.0560
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9873 - loss: 0.0414 - val_accuracy: 0.9789 - val_loss: 0.0807
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9875 - loss: 0.0405 - val_accuracy: 0.9805 - val_loss: 0.0767
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9781 - loss: 0.0877
Test accuracy: 0.9804999828338623


In [10]:
!airflow db upgrade

/home/aiffel04/my_project/aiffel/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name
/home/aiffel04/my_project/aiffel/lib/python3.12/site-packages/airflow/cli/cli_config.py:49 DeprecationWarning: `db upgrade` is deprecated. Use `db migrate` instead.
DB: sqlite:////home/aiffel04/airflow/airflow.db
Performing upgrade to the metadata database sqlite:////home/aiffel04/airflow/airflow.db
[2024-11-12T14:43:13.979+0900] {migration.py:207} INFO - Context impl SQLiteImpl.
[2024-11-12T14:43:13.980+0900] {migration.py:210} INFO - Will assume non-transactional DDL.
[2024-11-12T14:43:13.981+0900] {migration.py:207} INFO - Context impl SQLiteImpl.
[2024-11-12T14:43:13.981+0900] {migration.py:210} INFO - Will assume non-transactional DDL.
[2024-11-12T14:43:13.982+0900] {db.py:1675} INFO - Creating tables
INFO  [alembic

In [22]:
!airflow db reset
!airflow db init

/home/aiffel04/my_project/aiffel/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name
DB: sqlite:////home/aiffel04/airflow/airflow.db
This will drop existing tables if they exist. Proceed? (y/n)^C
Traceback (most recent call last):
  File "/home/aiffel04/my_project/aiffel/bin/airflow", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/home/aiffel04/my_project/aiffel/lib/python3.12/site-packages/airflow/__main__.py", line 62, in main
    args.func(args)
  File "/home/aiffel04/my_project/aiffel/lib/python3.12/site-packages/airflow/cli/cli_config.py", line 49, in command
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/aiffel04/my_project/aiffel/lib/python3.12/site-packages/airflow/utils/providers_configuration_loader.py", line 55, in wrapped_function
    re

In [16]:
# terminal에 서 실 행 함
#airflow users create \
    #--username jungseoyeon \
    #--firstname 정 \
    #--lastname 서연 \
    #--role Admin \
    #--email jbw8715@naver.com


In [23]:
execution_date = datetime(2024, 11, 10, tzinfo=local_tz)
dag.test(execution_date=execution_date)

[2024-11-12T11:41:47.578+0900] {dag.py:4435} INFO - dagrun id: mnist_training_pipeline
[2024-11-12T11:41:47.588+0900] {dag.py:4451} INFO - created dagrun <DagRun mnist_training_pipeline @ 2024-11-10 00:00:00+09:00: manual__2024-11-10T00:00:00+09:00, state:running, queued_at: None. externally triggered: False>
[2024-11-12T11:41:47.608+0900] {dag.py:4396} INFO - [DAG TEST] starting task_id=load_and_preprocess_data map_index=-1
[2024-11-12T11:41:47.609+0900] {dag.py:4399} INFO - [DAG TEST] running task <TaskInstance: mnist_training_pipeline.load_and_preprocess_data manual__2024-11-10T00:00:00+09:00 [scheduled]>
[2024-11-12T11:41:47.623+0900] {workday.py:41} WARNING - Could not import pandas. Holidays will not be considered.
[2024-11-12 11:41:47,678] {taskinstance.py:3132} INFO - Exporting env vars: AIRFLOW_CTX_DAG_OWNER='airflow' AIRFLOW_CTX_DAG_ID='mnist_training_pipeline' AIRFLOW_CTX_TASK_ID='load_and_preprocess_data' AIRFLOW_CTX_EXECUTION_DATE='2024-11-09T15:00:00+00:00' AIRFLOW_CTX_TR

<DagRun mnist_training_pipeline @ 2024-11-09 15:00:00+00:00: manual__2024-11-10T00:00:00+09:00, state:failed, queued_at: None. externally triggered: False>

In [ ]:
import wandb
from datetime import datetime
import numpy as np

def load_and_preprocess_data():
    # MNIST 데이터셋 로드 및 전처리
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    x_train = x_train[..., np.newaxis]
    x_test = x_test[..., np.newaxis]
    
    # JSON 직렬화를 위해 리스트로 변환
    return {
        "x_train": x_train.tolist(),
        "y_train": y_train.tolist(),
        "x_test": x_test.tolist(),
        "y_test": y_test.tolist()
    }

def train_model(**kwargs):
    """모델 학습 및 W&B 로깅"""
    
    # W&B 초기화
    wandb.init(
        project="mnist_classification",
        config={
            "learning_rate": 0.001,
            "conv1_filters": 32,
            "conv2_filters": 64,
            "epochs": 5,
            "batch_size": 64
        }
    )
    
    # 데이터 로드: Airflow 실행 시 XCom에서 데이터 가져오기
    if 'ti' in kwargs:
        data = kwargs['ti'].xcom_pull(task_ids='load_and_preprocess_data')
        x_train = np.array(data['x_train'])
        y_train = np.array(data['y_train'])
        x_test = np.array(data['x_test'])
        y_test = np.array(data['y_test'])
    else:
        # Jupyter Notebook에서 실행될 때는 데이터를 직접 로드
        data = load_and_preprocess_data()
        x_train = np.array(data['x_train'])
        y_train = np.array(data['y_train'])
        x_test = np.array(data['x_test'])
        y_test = np.array(data['y_test'])
    
    # 모델 생성
    config = wandb.config
    model = create_model(
        learning_rate=config.learning_rate,
        conv1_filters=config.conv1_filters,
        conv2_filters=config.conv2_filters
    )
    
    # 모델 학습
    history = model.fit(
        x_train, y_train,
        epochs=config.epochs,
        batch_size=config.batch_size,
        validation_data=(x_test, y_test),
        callbacks=[wandb.keras.WandbCallback(save_graph=False, save_model=False)]
    )

    # 모델 평가
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {test_accuracy}")
    
    # W&B에 최종 메트릭 기록
    wandb.log({
        "test_loss": test_loss,
        "test_accuracy": test_accuracy
    })
    
    # 모델 저장
    model.save(f"mnist_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.keras")
    wandb.finish()


# 하이퍼파라미터 튜닝 함수
def hyperparameter_sweep():
    sweep_config = {
        'method': 'random',
        'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
        'parameters': {
            'learning_rate': {'values': [0.001, 0.01, 0.0001]},
            'conv1_filters': {'values': [16, 32, 64]},
            'conv2_filters': {'values': [32, 64, 128]},
            'batch_size': {'values': [64, 128, 256]}
        }
    }
    
    sweep_id = wandb.sweep(sweep_config, project="mnist_classification")
    wandb.agent(sweep_id, train_model, count=5)

# Airflow DAG 정의
local_tz = pendulum.timezone("Asia/Seoul")

default_args = {
    'owner': 'airflow',
    'start_date': datetime(2024, 1, 1, tzinfo=local_tz),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'mnist_training_pipeline',
    default_args=default_args,
    description='MNIST 학습 파이프라인',
    schedule_interval='@daily',
    catchup=False
)

# DAG 태스크 정의
preprocessing_task = PythonOperator(
    task_id='load_and_preprocess_data',
    python_callable=load_and_preprocess_data,
    dag=dag
)

training_task = PythonOperator(
    task_id='train_model',
    python_callable=train_model,
    dag=dag
)

hyperparameter_tuning_task = PythonOperator(
    task_id='hyperparameter_tuning',
    python_callable=hyperparameter_sweep,
    dag=dag
)

# 태스크 의존성 설정
preprocessing_task >> training_task >> hyperparameter_tuning_task

# Jupyter Notebook에서 직접 실행하기 위한 코드
if __name__ == "__main__":
    train_model()
    hyperparameter_sweep()


/tmp/ipykernel_270309/1951795695.py:109 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

wandb: WARNING Ignored wandb.init() arg project when running a sweep.
